In [1]:
# Imports
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [2]:
import numpy as np

**Standard Relu**

In [3]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc2 = nn.Linear(250,250)
        self.fc3 = nn.Linear(250, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x)) # replace this with siren.
        x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x


In [4]:
device = torch.device('cuda')

In [5]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 3e-4 # karpathy's constant
batch_size = 64
num_epochs = 3

In [6]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 233515000.56it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 101453679.92it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 75361135.41it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16858875.02it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [7]:
# Initialize network
model_relu = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_relu.parameters(), lr=learning_rate)

In [8]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_relu(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:10<00:00, 89.19it/s] 


In [9]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model_relu)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_relu)*100:.2f}")

Accuracy on training set: 96.97
Accuracy on test set: 96.89


In [10]:
def compute_gradient (func,inp, **kwargs):
    inp.requires_grad = True
    loss = func(inp, **kwargs)
    loss.backward()
    inp.requires_grad = False
    return inp.grad.data

In [11]:
def func(inp, model = None, target = None):
    out = model(inp)
    loss = torch.nn.functional.nll_loss(out,torch.LongTensor([target]).to(device))
    #print(f"Loss:  {loss.item()}")
    return loss

In [12]:
def attack (tensor, model, eps = 1e-3, n_iter = 5000):
    number = 5000
    new_tensor = tensor.unsqueeze(0).detach().clone()
    new_tensor = new_tensor.to(device = device)
    orig_prediction = model(new_tensor).argmax()
    #print(f"Original Prediction: {orig_prediction.item()}")

    for i in tqdm(range(n_iter)):
        model.zero_grad()

        grad = compute_gradient(func, new_tensor, model = model, target = orig_prediction.item())
        #new_tensor = torch.clamp(new_tensor + eps * grad.sign(), -2, 2)
        new_tensor = new_tensor + eps * grad.sign()
        new_prediction = model(new_tensor).argmax()

        if new_prediction != orig_prediction:
            #print(f"New Prediction: {new_prediction.item()}")
            number = i
            break
    return number


In [13]:
num1 = attack(train_dataset[0][0], model_relu)
num1

  1%|          | 48/5000 [00:00<00:27, 183.10it/s]


48

In [14]:
# now its time for siren.
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.

    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a
    # hyperparameter.

    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)

    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30,is_linear = False):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.is_linear = is_linear
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)

    def forward(self, input):
        if self.is_linear:
          return (self.omega_0 * self.linear(input))
        return torch.sin(self.omega_0 * self.linear(input))

    def forward_with_intermediate(self, input):
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate


class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False,
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()

        self.net = []
        self.net.append(SineLayer(in_features, hidden_features,
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features,
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)

            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0,
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)

            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features,
                                      is_first=False, omega_0=hidden_omega_0))

        self.net = nn.Sequential(*self.net)

    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords

    def forward_with_activations(self, coords, retain_grad=False):
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations['input'] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)

                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()

                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else:
                x = layer(x)

                if retain_grad:
                    x.retain_grad()

            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

In [15]:
# Simple CNN
class siren_CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(siren_CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        #self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc1 = SineLayer(16 * 7 * 7, 250, is_first=True, omega_0=30)
        self.fc2 = SineLayer(250, 250, is_first=False, omega_0=30)
        self.fc3 = nn.Linear(250,num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        #x = F.relu(self.fc1(x)) # replace this with siren.
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [16]:
# Initialize network
model_siren = siren_CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_siren.parameters(), lr=learning_rate)

In [17]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_siren(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:09<00:00, 96.21it/s] 


In [18]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model_siren)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_siren)*100:.2f}")

Accuracy on training set: 98.55
Accuracy on test set: 97.85


In [19]:
num1 = attack(train_dataset[0][0], model_siren)
num1

  1%|          | 45/5000 [00:00<00:11, 446.84it/s]


45

In [22]:
# Simple CNN
class siren_updated_CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(siren_updated_CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        #self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc1 = SineLayer(16 * 7 * 7, 250, is_first=True, omega_0=30)
        self.fc2 = SineLayer(250, 250, is_first=False, omega_0=30)
        self.fc3 = SineLayer(250,num_classes, is_first = False)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        #x = F.relu(self.fc1(x)) # replace this with siren.
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [23]:
# Initialize network
model_siren_updated = siren_updated_CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_siren_updated.parameters(), lr=learning_rate)

In [24]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_siren_updated(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:09<00:00, 100.65it/s]


In [25]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model_siren_updated)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_siren_updated)*100:.2f}")

Accuracy on training set: 98.61
Accuracy on test set: 98.09


In [26]:
num1 = attack(train_dataset[0][0], model_siren_updated)
num1

  0%|          | 14/5000 [00:00<00:14, 335.78it/s]


14

In [27]:
# Simple CNN
class relu_updated_CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(relu_updated_CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc2 = nn.Linear(250,250)
        self.fc3 = nn.Linear(250, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x)) # replace this with siren.
        x = F.relu(self.fc2(x))
        x = self.fc2(x)
        x = torch.sin(x)
        return x


In [28]:
# Initialize network
model_relu_updated = relu_updated_CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_relu_updated.parameters(), lr=learning_rate)

In [29]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_relu_updated(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:09<00:00, 101.46it/s]


In [30]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model_relu_updated)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_relu_updated)*100:.2f}")

Accuracy on training set: 91.81
Accuracy on test set: 91.75


In [31]:
num1 = attack(train_dataset[0][0], model_relu_updated)
num1

  0%|          | 25/5000 [00:00<00:12, 399.38it/s]


25

In [32]:
def compute_avg_n_iter (model):
    sum = 0
    for i in tqdm(range(1000)):
        sum += attack(train_dataset[i][0], model)
    return sum/1000

In [33]:
def new_compute_avg_n_iter (model):
    sum = 0
    counter = 0
    i = 0
    while(counter<1000):
        siren_prediction = model_siren(train_dataset[i][0].unsqueeze(0).to(device)).argmax()
        relu_prediction = model_relu(train_dataset[i][0].unsqueeze(0).to(device)).argmax()
        actual = train_dataset[i][1]
        if (siren_prediction == actual and relu_prediction == actual):
            sum += attack(train_dataset[i][0], model)
            counter += 1
        i += 1
    return sum/1000

In [34]:
def new_compute_avg_n_iter_test (model_1, model_2):
    sum_1 = 0
    sum_2 = 0
    counter = 0
    i = 0
    while(counter<100):
        model_1_prediction = model_1(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        model_2_prediction = model_2(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        actual = test_dataset[i][1]
        print(counter)
        if (model_1_prediction == actual and model_2_prediction == actual):
            sum_1 += attack(test_dataset[i][0], model_1)
            sum_2 += attack(test_dataset[i][0], model_2)
            counter += 1
        i += 1
    return sum_1/100, sum_2/100

In [35]:
print(f"avg n_iter for model_relu and model_siren = {new_compute_avg_n_iter_test(model_relu, model_siren)}")

0


  2%|▏         | 85/5000 [00:00<00:11, 445.56it/s]


1


  1%|▏         | 71/5000 [00:00<00:11, 429.48it/s]


2


...

98


  2%|▏         | 79/5000 [00:00<00:11, 443.63it/s]


99


  1%|          | 39/5000 [00:00<00:11, 449.42it/s]

avg n_iter for model_relu and model_siren = (108.8, 74.36)


In [36]:
print(f"avg n_iter for model_relu and model_siren_updated = {new_compute_avg_n_iter_test(model_relu, model_siren_updated)}")

0


  5%|▌         | 266/5000 [00:00<00:11, 426.73it/s]


1


  3%|▎         | 131/5000 [00:00<00:11, 430.89it/s]


2


...

98


  5%|▌         | 273/5000 [00:01<00:19, 244.74it/s]


99


100%|██████████| 5000/5000 [00:12<00:00, 385.38it/s]

avg n_iter for model_relu and model_siren_updated = (111.13, 2082.4)


In [37]:
print(f"avg n_iter for model_relu and model_relu_updated = {new_compute_avg_n_iter_test(model_relu, model_relu_updated)}")

0


  3%|▎         | 155/5000 [00:00<00:09, 493.62it/s]


1


  3%|▎         | 159/5000 [00:00<00:10, 477.75it/s]


2


...

98


  1%|▏         | 68/5000 [00:00<00:10, 454.66it/s]


99


  1%|          | 53/5000 [00:00<00:10, 452.91it/s]

avg n_iter for model_relu and model_relu_updated = (111.62, 117.44)


In [38]:
def attack_cross (tensor, model, eps = 1e-3, n_iter = 5000):
    number = 5000
    new_tensor = tensor.unsqueeze(0).detach().clone()
    new_tensor = new_tensor.to(device = device)
    orig_prediction = model(new_tensor).argmax()
    #print(f"Original Prediction: {orig_prediction.item()}")

    for i in tqdm(range(n_iter)):
        model.zero_grad()

        grad = compute_gradient(func, new_tensor, model = model, target = orig_prediction.item())
        #new_tensor = torch.clamp(new_tensor + eps * grad.sign(), -2, 2)
        new_tensor = new_tensor + eps * grad.sign()
        new_prediction = model(new_tensor).argmax()

        if new_prediction != orig_prediction:
            #print(f"New Prediction: {new_prediction.item()}")
            number = i
            break
    return number, new_tensor

In [39]:
def new_compute_avg_n_iter_test_cross (model_1, model_2):
    sum_1 = 0
    sum_2 = 0
    correct_predictions_1 = 0
    correct_predictions_2 = 0
    counter = 0
    i = 0
    while(counter<100):
        model_1_prediction = model_1(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        model_2_prediction = model_2(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        actual = test_dataset[i][1]
        print(counter)
        if (model_1_prediction == actual and model_2_prediction == actual):
            val_1, val_2 = attack_cross(test_dataset[i][0], model_1)
            sum_1 += val_1
            adv_example_from_model_1 = val_2
            adv_example_from_model_1_on_model_2_prediction = model_2(adv_example_from_model_1).argmax()
            if (adv_example_from_model_1_on_model_2_prediction == actual):
              correct_predictions_2 += 1
            val_1, val_2 = attack_cross(test_dataset[i][0], model_2)
            sum_2 += val_1
            adv_example_from_model_2 = val_2
            adv_example_from_model_2_on_model_1_prediction = model_1(adv_example_from_model_2).argmax()
            if (adv_example_from_model_2_on_model_1_prediction == actual):
              correct_predictions_1 += 1

            counter += 1
        i += 1
    return sum_1/100, sum_2/100, correct_predictions_1, correct_predictions_2

In [40]:
new_compute_avg_n_iter_test_cross(model_relu, model_siren_updated)

0


  5%|▌         | 266/5000 [00:00<00:10, 444.36it/s]


1


  3%|▎         | 131/5000 [00:00<00:11, 419.65it/s]


2


...

98


  5%|▌         | 273/5000 [00:00<00:12, 391.89it/s]


99


100%|██████████| 5000/5000 [00:11<00:00, 420.40it/s]


(111.13, 2082.4, 97, 99)

In [ ]:
print(f"avg n_iter for model_relu and model_siren = {new_compute_avg_n_iter_test(model_relu, model_siren)}")

In [ ]:
print(f"avg n_iter for model_siren = {new_compute_avg_n_iter_test(model_siren)}")

In [ ]:
print(f"avg n_iter for model_siren_updated = {new_compute_avg_n_iter_test(model_siren_updated)}")

In [ ]:
print(f"avg n_iter for model_relu_updated = {new_compute_avg_n_iter_test(model_relu_updated)}")